# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

Este notebook se ha ejecutado usando el dataset de _BelgiumTS_
https://btsd.ethz.ch/shareddata/

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)
print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
labels_path = os.path.join(ROOT_PATH, "dataset_bel/labels.csv")
train_path = os.path.join(ROOT_PATH, "dataset_bel/train")
test_path = os.path.join(ROOT_PATH, "dataset_bel/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [3]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)

# Recuperamos los nombres de las categorias. Los diferentes tipo de señales
# que se van a clasificar.
titles = traffic.read_csv(labels_path, ",")

# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")
print("Titles total: ", len(titles))
traffic.print_signals_attributes(10, images_train, labels_train, titles)

Total images (train):  4575
Total labels (train):  62
Images shape:  (4575, 64, 64, 3)
Labels shape:  (4575,)
Total images (test):  2520
Total labels (test):  53
Images shape:  (2520, 64, 64, 3)
Labels shape:  (2520,)
Titles total:  62
Signal:  A31
shape:  (64, 64, 3) 	min: 0.04844037224264742 	max:  0.9921568627450981
shape:  (64, 64, 3) 	min: 0.012683823529412207 	max:  1.0
shape:  (64, 64, 3) 	min: 0.03416077856924062 	max:  0.7854355755974258
shape:  (64, 64, 3) 	min: 0.0391180300245094 	max:  0.8150242225796563
shape:  (64, 64, 3) 	min: 0.0064950980392159095 	max:  0.996078431372549
shape:  (64, 64, 3) 	min: 0.06834022671568622 	max:  0.8909658394607853
shape:  (64, 64, 3) 	min: 0.024230238970588236 	max:  0.9842294730392157
shape:  (64, 64, 3) 	min: 0.04133013556985346 	max:  1.0
shape:  (64, 64, 3) 	min: 0.023467419194240183 	max:  0.7593352673100491
shape:  (64, 64, 3) 	min: 0.01371017156862504 	max:  0.9921568627450981
shape:  (64, 64, 3) 	min: 0.07595454197303977 	max:  0.756

In [4]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
def get_keras_model():
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation='relu', input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [6]:
model = get_keras_model()

# Muestra la arquitectura de nuestra red neuronal
model.summary()

# Configurando el modelo de aprendijaze:
#  · loss, función para evaluar el grado de error entre salidas calculadas
#  · optimizador, función para calcular los pesos de los parámetros a partir de los datos de entrada
#  · metricas, para monitorizar el proceso de aprendizaje de la red.
model.compile(loss="categorical_crossentropy",
             optimizer="sgd",
             metrics=['accuracy'])

# Entrenamiento del modelo
# - batch_size, indica el número de datos que se usan en cada actualización.
# - epochs, indica el número de veces que se usan todos los datos del proceso.
#model.fit(np_images_train, labels_categorical_train, batch_size=32, epochs=20)
model.fit(np_images_train, labels_categorical_train,
          batch_size = 32,
          epochs = 30,
          callbacks=[ModelCheckpoint('model_64_64_bel.h5', save_best_only = False)])

# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)  

In [7]:
# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

2520/2520 [==============================] - 3s 1ms/step
Test loss: 0.174172989903768
Test accuracy: 0.9662698412698413


In [8]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.,)

datagen.fit(np_images_test)

In [9]:
model_2 = get_keras_model()

model_2.compile(loss="categorical_crossentropy",
             optimizer="sgd",
             metrics=['accuracy'])

model_2.fit_generator(datagen.flow(np_images_train, labels_categorical_train, batch_size=32),
                      steps_per_epoch = np_images_train.shape[0],
                      epochs = 30,
                      callbacks=[ModelCheckpoint('model_data_aug_64_64_bel.h5',save_best_only = False)])

Epoch 1/30
4575/4575 [==============================] - 755s 165ms/step - loss: 1.0304 - acc: 0.7274
Epoch 2/30
4575/4575 [==============================] - 752s 164ms/step - loss: 0.2244 - acc: 0.9343
Epoch 3/30
4575/4575 [==============================] - 749s 164ms/step - loss: 0.1233 - acc: 0.9637
Epoch 4/30
4575/4575 [==============================] - 752s 164ms/step - loss: 0.0841 - acc: 0.9747
Epoch 5/30
4575/4575 [==============================] - 753s 165ms/step - loss: 0.0624 - acc: 0.9813
Epoch 6/30
4575/4575 [==============================] - 747s 163ms/step - loss: 0.0494 - acc: 0.9852
Epoch 7/30
4575/4575 [==============================] - 747s 163ms/step - loss: 0.0426 - acc: 0.9869
Epoch 8/30
4575/4575 [==============================] - 748s 163ms/step - loss: 0.0348 - acc: 0.9895
Epoch 9/30
4575/4575 [==============================] - 751s 164ms/step - loss: 0.0311 - acc: 0.9903
Epoch 10/30
4575/4575 [==============================] - 779s 170ms/step - loss: 0.0283 - a

In [10]:
# Evaluación del modelo
test_loss, test_acc = model_2.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

y_pred = model_2.predict_classes(np_images_test)
acc = np.sum(y_pred == np_labels_test) / np.size(np_labels_test)
print("Test accuracy: {}".format(acc))

2520/2520 [==============================] - 4s 1ms/step
Test loss: 0.08093734560057161
Test accuracy: 0.9841269841269841
Test accuracy: 0.9841269841269841
